In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir('/content/drive/My Drive/')

In [0]:
os.chdir('ir_project/New folder/')

In [4]:
'test_data.npy' in os.listdir('.')

True

In [5]:
!pip install transformers

In [0]:
import transformers

In [0]:
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')

In [8]:
import transformers
import pandas as pd
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

from torch.utils import data
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
all_labels=[]
train_data=np.load('train_data.npy',allow_pickle=True)
for row in train_data:
    if row[-1]=='':
        continue
    else:
        all_labels.extend(row[-1]['category'])

all_labels=list(set(all_labels))

print("number of classes = ",len(all_labels))

labels_mapping={}

idx=0
labels_mapping['NIL']=0
for label in all_labels:
    idx+=1
    labels_mapping[label]=idx

x_train=[]
y_train=[]
for row in train_data:
    sentence = row[2]
    
    x_train.append(sentence)
    
    if row[-1]=='':
        y_train.append([0])
        continue
    a=[]
    labels = row[-1]['category']
    for i in labels:
        
        a.append(labels_mapping[i])
    y_train.append(a)

tokenized_sentences = []
labels=[]

for (s,l) in zip(x_train,y_train) :
    if isinstance(s,float):
        continue
    a=tokenizer.encode(s,add_sepcial_tokens=True)
    tokenized_sentences.append(a)
    labels.append(l)

max_len = 0
for i in tokenized_sentences:
    if len(i) > max_len:
        max_len = len(i)


padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized_sentences])
input_ids = torch.LongTensor(np.array(padded))
attention_mask = np.where(padded != 0, 1, 0)

'''
with torch.no_grad():
    last_hidden_states = model(input_ids)
'''

class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.model_class, self.tokenizer_class, self.pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')
        self.tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
        self.model = model_class.from_pretrained(pretrained_weights)
        
        self.drop_out=nn.Dropout(0.1)
        self.conv1=nn.Conv1d(768*2,768,1)
        
        self.output=nn.Linear(768,82)
    
    def forward(self,x,attention_masks):
        
        x = self.model(x,attention_masks)
        #print(x[0].shape)
        x = torch.cat((x[0][:,-1,:],x[0][:,-2,:]),dim=-1)
        #print(x.shape)
        x = x.unsqueeze(2)
        #print(x.shape)
        x=self.drop_out(x)
        
        x=self.conv1(x)
        #print(x.shape)
        x=F.relu(x)
        #print(x.shape)
        
        x=x.squeeze(2)
        #print(x.shape)
        x=self.output(x)
        #print(x.shape)
        return x
    
class TargetLoader(data.Dataset):

  def __init__(self,sentences,new_y_train,attention_mask):

    self.sentences=sentences
    self.new_y_train=new_y_train
    self.attention_mask=attention_mask
    
    self.length=len(self.sentences)


  def __getitem__(self,idx):

    image=self.sentences[idx]
    label=self.new_y_train[idx]
    mask = self.attention_mask[idx]
    return image,label,mask
  
  def __len__(self):

    return self.length


new_y_train=[]
#https://discuss.pytorch.org/t/what-kind-of-loss-is-better-to-use-in-multilabel-classification/32203/4

for label in y_train:
    a=torch.LongTensor(label)
    a=a.unsqueeze(0)
    target = torch.zeros(a.size(0),82).scatter_(1, a, 1.)
    new_y_train.append(target.numpy()[0])

new_y_train=torch.FloatTensor(np.array(new_y_train))


train_target=TargetLoader(input_ids,new_y_train,attention_mask)

train_loader=torch.utils.data.DataLoader(train_target,batch_size=16,shuffle=True)

net = Model()
net = net.cuda()
optimizer = optim.Adadelta(net.parameters(), lr=1.0, rho=0.95, eps=1e-06, weight_decay=0)
criterion=nn.BCEWithLogitsLoss()

for epoch in range(15):
    
    total_loss=0
    for sentences,labels,mask in train_loader:
        #break
        sentences = sentences.cuda()
        labels = labels.cuda()
        mask=mask.cuda()
        preds = net(sentences,mask)
        
        loss = criterion(preds,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #break
        total_loss+=loss.item()
        
        print("loss :",loss.item())
    print("epoch done !",epoch, " loss : ",total_loss)

number of classes =  81
loss : 0.686030924320221
loss : 0.6333944201469421
loss : 0.5419178009033203
loss : 0.36799490451812744
loss : 0.2833956778049469
loss : 0.08387395739555359
loss : 0.06588933616876602
loss : 0.06877664476633072
loss : 0.06592719256877899
loss : 0.07223622500896454
loss : 0.06904394179582596
loss : 0.06102750077843666
loss : 0.07113512605428696
loss : 0.062367234379053116
loss : 0.06849164515733719
loss : 0.06352962553501129
loss : 0.0663682296872139
loss : 0.0614696741104126
loss : 0.0674188956618309
loss : 0.06610897928476334
loss : 0.07582955062389374
loss : 0.05743352696299553
loss : 0.06381037831306458
loss : 0.0526198111474514
loss : 0.05293043330311775
loss : 0.05869261547923088
loss : 0.054453760385513306
loss : 0.0727647989988327
loss : 0.07284417748451233
loss : 0.05234520509839058
loss : 0.07233911007642746
loss : 0.07580987364053726
loss : 0.07280539721250534
loss : 0.06659400463104248
loss : 0.05474301055073738
loss : 0.04686954617500305
loss : 0.076

In [9]:
predictions=[]
gt=[]

for sentences,labels,mask in train_loader:
    sentences=sentences.cuda()
    mask=mask.cuda()
    preds=net(sentences,mask)
    #print(preds.shape)
    preds=F.sigmoid(preds)
    #print(preds.shape)
    
    labels_=labels.cpu().detach().numpy().flatten()
    gt.extend(labels_)
    
    preds=preds.cpu().detach().numpy().flatten()
    
    for p in preds:
        
        if p>0.2:
            predictions.append(1)
        else:
            predictions.append(0)

gt=np.array(gt)
predictions = np.array(predictions)
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

c=classification_report(gt,predictions)

print(c.split('\n'))


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


['              precision    recall  f1-score   support', '', '         0.0       1.00      1.00      1.00    140286', '         1.0       0.97      0.99      0.98      2312', '', '    accuracy                           1.00    142598', '   macro avg       0.98      1.00      0.99    142598', 'weighted avg       1.00      1.00      1.00    142598', '']


In [10]:
c.split('\n')

['              precision    recall  f1-score   support',
 '',
 '         0.0       1.00      1.00      1.00    140286',
 '         1.0       0.97      0.99      0.98      2312',
 '',
 '    accuracy                           1.00    142598',
 '   macro avg       0.98      1.00      0.99    142598',
 'weighted avg       1.00      1.00      1.00    142598',
 '']

In [11]:
all_labels=[]
test_data=np.load('test_data.npy',allow_pickle=True)
for row in test_data:
    if row[-1]=='':
        continue
    else:
        all_labels.extend(row[-1]['category'])

all_labels=list(set(all_labels))
print(all_labels)
print("number of classes = ",len(all_labels))

x_test=[]
y_test=[]
for row in test_data:
    sentence = row[2]
    
    x_test.append(sentence)
    
    if row[-1]=='':
        y_test.append([0])
        continue
    a=[]
    labels = row[-1]['category']
    for i in labels:
        if i not in labels_mapping.keys():
          a.append(0)
        else:
          a.append(labels_mapping[i])
    y_test.append(a)

tokenized_sentences = []
labels=[]

for (s,l) in zip(x_test,y_test) :
    if isinstance(s,float):
        continue
    b=tokenizer.encode(s,add_sepcial_tokens=True)
    tokenized_sentences.append(b)
    labels.append(l)

max_len = 0
for i in tokenized_sentences:
    if len(i) > max_len:
        max_len = len(i)


padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized_sentences])
input_ids = torch.LongTensor(np.array(padded))
attention_mask = np.where(padded != 0, 1, 0)

class TestLoader(data.Dataset):

  def __init__(self,sentences,new_y_test,y_test,attention_mask):

    self.sentences=sentences
    self.new_y_train=new_y_train
    self.attention_mask=attention_mask
    self.y_test=y_test
    self.length=len(self.sentences)


  def __getitem__(self,idx):

    image=self.sentences[idx]
    label=self.new_y_train[idx]
    mask = self.attention_mask[idx]
    original =self.y_test[idx]
    return image,label,mask,original
  
  def __len__(self):

    return self.length


new_y_test=[]
#https://discuss.pytorch.org/t/what-kind-of-loss-is-better-to-use-in-multilabel-classification/32203/4

for label in y_test:
    #print(label)
    a=torch.LongTensor(label)
    a=a.unsqueeze(0)
    #print(a)
    target = torch.zeros(a.size(0),82).scatter_(1, a, 1.)
    new_y_test.append(target.numpy()[0])

new_y_train=torch.FloatTensor(np.array(new_y_test))
print(len(input_ids))
print("y_test = ",y_test[0])

test_target=TestLoader(input_ids,new_y_test,y_test,attention_mask)

test_loader=torch.utils.data.DataLoader(test_target,batch_size=16,shuffle=True)

predictions=[]
gt=[]
o_labels=[]
o_pred=[]
index=0
for sentences,labels,mask,original in test_loader:
    sentences=sentences.cuda()
    mask=mask.cuda()
    preds=net(sentences,mask)
    #print(preds.shape)
    preds=F.sigmoid(preds)
    #print(preds.shape)
    
    labels_=labels.cpu().detach().numpy().flatten()
    gt.extend(labels_)
  
    preds=preds.cpu().detach().numpy().flatten()
    original_pred=[]
    original_label=[]
    prediction=[]
    for p in preds:
        
        if p>0.2:
            prediction.append(1)
        else:
            prediction.append(0)
    predictions.extend(prediction)
    idx =0
    for one_label in labels:
      for k1 in range(len(one_label)):
        if one_label[k1]==0:
          original_label.append(0)
        else:
          original_label.append(k1)
        #print(prediction[k1])
        if prediction[idx]==0:
          original_pred.append(0)
        else:
          original_pred.append(k1)
        idx+=1
      o_labels.extend(original_label)
      o_pred.extend(original_pred)

    index+=1
  
gt=np.array(gt)
predictions = np.array(predictions)
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

c1=classification_report(gt,predictions)

gt=np.array(o_labels)
predictions = np.array(o_pred)
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

c2=classification_report(gt,predictions)
print(c2.split('\n'))


['MEMORY#DESIGN_FEATURES', 'LAPTOP#PRICE', 'CPU#OPERATION_PERFORMANCE', 'KEYBOARD#USABILITY', 'MOUSE#USABILITY', 'HARD_DISC#QUALITY', 'GRAPHICS#GENERAL', 'HARDWARE#GENERAL', 'DISPLAY#USABILITY', 'HARDWARE#OPERATION_PERFORMANCE', 'DISPLAY#GENERAL', 'LAPTOP#MISCELLANEOUS', 'LAPTOP#QUALITY', 'LAPTOP#USABILITY', 'MOUSE#OPERATION_PERFORMANCE', 'KEYBOARD#DESIGN_FEATURES', 'DISPLAY#QUALITY', 'SHIPPING#QUALITY', 'FANS_COOLING#QUALITY', 'LAPTOP#DESIGN_FEATURES', 'MOUSE#GENERAL', 'COMPANY#GENERAL', 'SUPPORT#QUALITY', 'BATTERY#QUALITY', 'HARDWARE#QUALITY', 'MULTIMEDIA_DEVICES#GENERAL', 'BATTERY#OPERATION_PERFORMANCE', 'OS#USABILITY', 'DISPLAY#OPERATION_PERFORMANCE', 'BATTERY#DESIGN_FEATURES', 'LAPTOP#OPERATION_PERFORMANCE', 'KEYBOARD#OPERATION_PERFORMANCE', 'LAPTOP#GENERAL', 'LAPTOP#CONNECTIVITY', 'BATTERY#GENERAL', 'OS#OPERATION_PERFORMANCE', 'MULTIMEDIA_DEVICES#USABILITY', 'GRAPHICS#QUALITY', 'OPTICAL_DRIVES#OPERATION_PERFORMANCE', 'SUPPORT#GENERAL', 'DISPLAY#DESIGN_FEATURES', 'SOFTWARE#GENERAL

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


['              precision    recall  f1-score   support', '', '           0       0.99      1.00      0.99    519887', '           1       1.00      1.00      1.00         7', '           2       1.00      0.21      0.35       118', '           3       0.94      0.81      0.87       521', '           4       0.00      0.00      0.00        17', '           5       0.62      0.23      0.34        77', '           6       0.67      0.73      0.70        95', '           7       0.38      0.70      0.49        33', '          11       0.00      0.00      0.00        11', '          15       0.64      0.56      0.60       108', '          16       0.78      0.36      0.49       611', '          17       0.65      0.53      0.58       608', '          19       0.52      0.45      0.48       368', '          21       0.52      0.53      0.53        87', '          23       0.55      0.52      0.54        82', '          24       0.63      0.74      0.68       192', '          25       0.00  

In [12]:
c1.split("\n")

['              precision    recall  f1-score   support',
 '',
 '         0.0       0.99      0.99      0.99     61337',
 '         1.0       0.65      0.57      0.61      1065',
 '',
 '    accuracy                           0.99     62402',
 '   macro avg       0.82      0.78      0.80     62402',
 'weighted avg       0.99      0.99      0.99     62402',
 '']

In [0]:
torch.save(net.state_dict(), "last_min.pt")

In [15]:
labels_mapping

{'BATTERY#GENERAL': 49,
 'BATTERY#MISCELLANEOUS': 61,
 'BATTERY#OPERATION_PERFORMANCE': 38,
 'BATTERY#QUALITY': 31,
 'COMPANY#GENERAL': 32,
 'CPU#MISCELLANEOUS': 78,
 'CPU#OPERATION_PERFORMANCE': 2,
 'CPU#QUALITY': 10,
 'DISPLAY#DESIGN_FEATURES': 55,
 'DISPLAY#GENERAL': 15,
 'DISPLAY#MISCELLANEOUS': 67,
 'DISPLAY#OPERATION_PERFORMANCE': 40,
 'DISPLAY#QUALITY': 24,
 'DISPLAY#USABILITY': 11,
 'FANS_COOLING#DESIGN_FEATURES': 58,
 'FANS_COOLING#OPERATION_PERFORMANCE': 22,
 'FANS_COOLING#QUALITY': 29,
 'GRAPHICS#DESIGN_FEATURES': 34,
 'GRAPHICS#GENERAL': 1,
 'GRAPHICS#MISCELLANEOUS': 63,
 'GRAPHICS#QUALITY': 51,
 'HARDWARE#QUALITY': 36,
 'HARD_DISC#DESIGN_FEATURES': 71,
 'HARD_DISC#QUALITY': 4,
 'KEYBOARD#DESIGN_FEATURES': 23,
 'KEYBOARD#GENERAL': 60,
 'KEYBOARD#OPERATION_PERFORMANCE': 45,
 'KEYBOARD#QUALITY': 70,
 'KEYBOARD#USABILITY': 5,
 'LAPTOP#CONNECTIVITY': 48,
 'LAPTOP#DESIGN_FEATURES': 30,
 'LAPTOP#GENERAL': 47,
 'LAPTOP#MISCELLANEOUS': 16,
 'LAPTOP#OPERATION_PERFORMANCE': 44,
 'LAP

In [14]:
c2.split('\n')

['              precision    recall  f1-score   support',
 '',
 '           0       0.99      1.00      0.99    519887',
 '           1       1.00      1.00      1.00         7',
 '           2       1.00      0.21      0.35       118',
 '           3       0.94      0.81      0.87       521',
 '           4       0.00      0.00      0.00        17',
 '           5       0.62      0.23      0.34        77',
 '           6       0.67      0.73      0.70        95',
 '           7       0.38      0.70      0.49        33',
 '          11       0.00      0.00      0.00        11',
 '          15       0.64      0.56      0.60       108',
 '          16       0.78      0.36      0.49       611',
 '          17       0.65      0.53      0.58       608',
 '          19       0.52      0.45      0.48       368',
 '          21       0.52      0.53      0.53        87',
 '          23       0.55      0.52      0.54        82',
 '          24       0.63      0.74      0.68       192',
 '       